In [1]:
import pandas as pd
from xgboost import XGBClassifier

# Load the model with the same parameters as in your training code
best_params = {'learning_rate': 0.2385173151321548, 'n_estimators': 59, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.8413689054381944, 'colsample_bytree': 0.9799902217451104, 'gamma': 0.002105660263980569, 'reg_alpha': 0.040797965666549, 'reg_lambda': 0.48241701342239474, 'scale_pos_weight': 38.39160162533456}
model = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')

# Define the features in the same order as used in training
features = [
    'Rank', 'PlayoffProgScore', 'FTA', 'DREB', 'AST/TO', 'REB%', 'CumulativePlayoffProgScore', 'AvgPIE_NextYearPlayers', 'Performance_NextYearCoach'
]

# Load and prepare the training data
dataset = pd.read_csv('../cleaned_data/dataset.csv')
X_train = dataset[features]
y_train = dataset['PlayoffNextSeason']

# Train the model on all training data
model.fit(X_train, y_train)

# Load year 10 data
year_10_data = pd.read_csv('../cleaned_data/year_10_stats.csv')

# Prepare the prediction data
X_pred = year_10_data[features]

# Get predictions and probabilities
predictions = model.predict(X_pred)
probabilities = model.predict_proba(X_pred)

# Create results DataFrame
results = pd.DataFrame({
    'Team': year_10_data['Team'],
    'Playoff_Prediction': predictions,
    'Playoff_Probability': probabilities[:, 1]  # Probability of positive class (making playoffs)
})

# Sort by probability in descending order
results = results.sort_values('Playoff_Probability', ascending=False)

# Display results
print("\nPlayoff Predictions for Next Season:")
print("=====================================")
for _, row in results.iterrows():
    print(f"{row['Team']}: {row['Playoff_Probability']*100:.2f}% chance of making playoffs")

# Save to CSV
results.to_csv('playoff_predictions_year_10.csv', index=False)
print("\nResults have been saved to 'playoff_predictions_year_10.csv'")


Playoff Predictions for Next Season:
LAS: 99.97% chance of making playoffs
SAS: 99.96% chance of making playoffs
WAS: 99.67% chance of making playoffs
PHO: 99.21% chance of making playoffs
CON: 98.99% chance of making playoffs
MIN: 92.39% chance of making playoffs
NYL: 89.19% chance of making playoffs
SEA: 87.57% chance of making playoffs
ATL: 84.03% chance of making playoffs
IND: 83.96% chance of making playoffs
TUL: 0.11% chance of making playoffs
CHI: 0.09% chance of making playoffs

Results have been saved to 'playoff_predictions_year_10.csv'
